In [1]:
import numpy
from scipy.spatial.transform import Rotation
import plotly.graph_objects as go

In [2]:
p0 = numpy.array([[-0.000570865656598318, 0.0884853366792768, -3.26546556240441e-05]]).flatten()
r0 = numpy.array([[1.20983339295868, -1.21200934113404, -1.20842711724355]]).flatten()
p1 = numpy.array([[-0.00061492213494309, 0.0884098065730318, 0.00010034292795414]]).flatten()
r1 = numpy.array([[1.21493087138343, -1.21216980419181, -1.20676702211217]]).flatten()
v = numpy.array([[-0.0138172156606369, -0.00465065861072804, 0.0229824747153206]]).flatten()
aabb_loose_min = numpy.array([[-0.0236770618485545, 0.102108852592854, -0.00489208277495219]]).flatten()
aabb_loose_max = numpy.array([[-0.0234331679048403, 0.102428078074496, -0.00451839770484339]]).flatten()

In [3]:
p0 = numpy.array([[-0.000570865656598318, 0.0884853366792768, -3.26546556240441e-05]]).flatten()
r0 = numpy.array([[1.20983339295868, -1.21200934113404, -1.20842711724355]]).flatten()
p1 = numpy.array([[-0.00061492213494309, 0.0884098065730318, 0.00010034292795414]]).flatten()
r1 = numpy.array([[1.21493087138343, -1.21216980419181, -1.20676702211217]]).flatten()
v = numpy.array([[-0.0138172156606369, -0.00465065861072804, 0.0229824747153206]]).flatten()
aabb_tight_min = numpy.array([[-0.0235920639499012, 0.102202015088203, -0.00475966672528616]]).flatten()
aabb_tight_max = numpy.array([[-0.023518054477777, 0.102334965325719, -0.00465068492399522]]).flatten()

In [13]:
p0 = numpy.array([[-0.000570865656598318, 0.0884853366792768, -3.26546556240441e-05]]).flatten()
r0 = numpy.array([[1.20983339295868, -1.21200934113404, -1.20842711724355]]).flatten()
p1 = numpy.array([[-0.00061492213494309, 0.0884098065730318, 0.00010034292795414]]).flatten()
r1 = numpy.array([[1.21493087138343, -1.21216980419181, -1.20676702211217]]).flatten()
v = numpy.array([[-0.0138172156606369, -0.00465065861072804, 0.0229824747153206]]).flatten()
aabb_tighter_min = numpy.array([[-0.0235835594457789, 0.102211340615431, -0.00474644685041807]]).flatten()
aabb_tighter_max = numpy.array([[-0.0235265384118029, 0.102325663319594, -0.0046639353535086]]).flatten()

In [19]:
dp = p1 - p0
dr = r1 - r0
dr

array([ 0.00509748, -0.00016046,  0.0016601 ])

In [15]:
def generate_line_search_trajectory(alpha):
    # Compute trajectory of a point
    v0_to_1 = [
        Rotation.from_rotvec(r0 + t * alpha * dr).as_matrix() @ v +
        (alpha * dp * t + p0) for t in numpy.linspace(0, 1, 1000)
    ]
    return numpy.vstack(v0_to_1)

In [16]:
def intervals_to_points(intervals):
    assert (len(intervals) == 3)
    return numpy.array([[intervals[0][i], intervals[1][j], intervals[2][k]]
                        for i in [0, 1] for j in [0, 1] for k in [0, 1]])


def cube(points, color):
    return go.Mesh3d(
        # 8 vertices of a cube
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        # i, j and k give the vertices of triangles
        i=[5, 0, 0, 0, 1, 1, 7, 7, 1, 0, 4, 6],
        j=[4, 1, 2, 6, 3, 7, 3, 6, 0, 2, 7, 4],
        k=[0, 5, 6, 4, 7, 5, 2, 2, 3, 3, 5, 7],
        color=color,
        flatshading=True)


def aabb_to_cube(aabb_min, aabb_max, color):
    return cube(
        intervals_to_points([[aabb_min[i], aabb_max[i]] for i in range(3)]),
        color)

In [17]:
data = []
for i in range(0, 1):
    points = generate_line_search_trajectory(1 / 2**i)
    data.append(
        go.Scatter3d(x=points[:, 0],
                     y=points[:, 1],
                     z=points[:, 2],
                     mode='lines',
                     name=f"$\\alpha=\\frac{{1}}{{2^{i}}}$"))
data.append(
    aabb_to_cube(aabb_loose_min, aabb_loose_max, "rgba(255, 0, 0, 0.05)"))
data.append(
    aabb_to_cube(aabb_tight_min, aabb_tight_max, "rgba(255, 255, 0, 0.05)"))
data.append(
    aabb_to_cube(aabb_tighter_min, aabb_tighter_max, "rgba(0, 255, 0, 0.05)"))
fig = go.Figure(data=data)
fig.show()